In [43]:
import cv2
from matplotlib import pyplot as plt
import mediapipe as mp
import numpy as np
import tensorflow as tf
import datetime


In [7]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
  image.flags.writeable = False                   # Image is no longer writeable
  results = model.process(image)                  # Make prediction
  image.flags.writeable = True                    # Image is now writeable
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR CONVERSION RGB 2 BGR
  return image, results

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [20]:
model = tf.keras.models.load_model('D:\Mini Project\Model\Model_with hello_thanks_iloveyou_dulaj\Action.h5')

In [22]:
actions = np.array(['hello', 'thanks', 'iloveyou'])

In [45]:
# 1. New detection variables
sentence = []
sequence = []
predictions = []
threshold = 0.7
val = ''


# Function to find the index of the first available camera
def find_available_camera():
    for i in range(10):  # Try the first 10 camera indices (adjust as needed)
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            cap.release()  # Release the camera immediately
            return i
    return None  # No available cameras found

# Find an available camera
camera_index = find_available_camera()

cap = cv2.VideoCapture(camera_index)

# Set mediopipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        #Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)

        #Draw Landmarks
        draw_styled_landmarks(image, results)

        #2. Prediction Logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
        sequence.append(keypoints)
        sequence = sequence[-29:]
#         print(len(sequence))

        if len(sequence) == 29:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            if (val != actions [np.argmax(res)]):
                print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                print(actions [np.argmax(res)])
            val = actions [np.argmax(res)]
            predictions.append(np.argmax(res))

        #3. Viz Logic
            if np.unique(predictions[-10:])[0]== np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image,''.join(sentence), (3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllwindows()

2023-09-28 12:56:32
hello
2023-09-28 12:56:37
thanks
2023-09-28 12:56:43
hello
2023-09-28 12:56:50
iloveyou
2023-09-28 12:56:52
hello
2023-09-28 12:56:54
iloveyou


AttributeError: module 'cv2' has no attribute 'destroyAllwindows'